In [6]:
import import_ipynb
from maze_common import Maze, shortestPathSearch, Cell
import time
import random as rnd
import numpy as np
import copy

In [33]:
maze = Maze(50, .3)
solvable = maze.isSolvable()
print("Is solvable?", solvable)
print("Obstacles?", maze.obstacles.shape[0])

Is solvable? True
Obstacles? 770


In [34]:
def manhattanDistance(cell, maze):
    return ((maze.dim - 1) - cell.row) + ((maze.dim - 1) - cell.col)

if (solvable):
    startTime = time.time()
#     print(maze.board)
    print(maze.obstacles.shape[0], "obstacles")
    shortestPath = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)

770 obstacles
***** Found shortest path in 0.03490447998046875 seconds *****
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (10, 8), (10, 9), (11, 9), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), (16, 10), (16, 11), (16, 12), (17, 12), (18, 12), (18, 13), (18, 14), (18, 15), (18, 16), (18, 17), (18, 18), (18, 19), (18, 20), (19, 20), (20, 20), (21, 20), (21, 21), (21, 22), (21, 23), (22, 23), (22, 24), (23, 24), (23, 25), (24, 25), (24, 26), (24, 27), (25, 27), (25, 28), (25, 29), (26, 29), (27, 29), (28, 29), (29, 29), (30, 29), (30, 30), (30, 31), (30, 32), (31, 32), (31, 33), (32, 33), (32, 34), (33, 34), (34, 34), (34, 35), (34, 36), (34, 37), (34, 38), (34, 39), (34, 40), (35, 40), (36, 40), (37, 40), (38, 40), (39, 40), (40, 40), (41, 40), (41, 41), (41, 42), (41, 43), (41, 44), (42, 44), (43, 44), (44, 44), (45, 44), (46, 44), (46, 45), (46, 46), (46, 47), (46, 48), (47, 48), 

In [36]:
def thinMaze(maze, fractionRemove):
    thinMaze = Maze(1, 0)
    thinMaze.board = copy.deepcopy(maze.board)
    thinMaze.obstacles = copy.deepcopy(maze.obstacles)
    thinMaze.dim = maze.dim
    numRemove = round(fractionRemove * thinMaze.obstacles.shape[0])
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX, obstacleY] = Cell.OPEN
    return thinMaze
    
thinnedMaze = thinMaze(maze, .5)
print("Original obstacles: ", maze.obstacles.shape[0])
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0])
# print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  770
Thinned obstacles:  385


In [37]:
visited = {}

def thinnedMazeShortestPathLength(cell, maze):
    shortestPathLength = 0
    if ((cell.row, cell.col)) not in visited:
        shortestPath = shortestPathSearch(thinnedMaze, startCoords = (cell.row, cell.col))
        shortestPathLength = len(shortestPath)
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cell.row, cell.col)]
    return shortestPathLength

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)

***** Found shortest path in 17.417726755142212 seconds *****
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (10, 8), (11, 8), (12, 8), (13, 8), (13, 9), (13, 10), (14, 10), (15, 10), (16, 10), (16, 11), (16, 12), (17, 12), (18, 12), (18, 13), (18, 14), (18, 15), (18, 16), (18, 17), (18, 18), (18, 19), (18, 20), (19, 20), (20, 20), (21, 20), (21, 21), (21, 22), (21, 23), (22, 23), (22, 24), (23, 24), (23, 25), (24, 25), (24, 26), (24, 27), (25, 27), (25, 28), (25, 29), (26, 29), (27, 29), (28, 29), (29, 29), (30, 29), (30, 30), (30, 31), (31, 31), (32, 31), (33, 31), (34, 31), (35, 31), (36, 31), (36, 32), (37, 32), (37, 33), (38, 33), (39, 33), (40, 33), (40, 34), (41, 34), (42, 34), (42, 35), (42, 36), (43, 36), (44, 36), (44, 37), (44, 38), (44, 39), (44, 40), (44, 41), (45, 41), (45, 42), (45, 43), (45, 44), (46, 44), (46, 45), (46, 46), (46, 47), (46, 48), (47, 48), (48, 48), (49, 48